## Example of GRACE

### Node-level representation learning on CORA

In [1]:
from dig.sslgraph.utils import Encoder
from dig.sslgraph.evaluation import NodeUnsupervised
from dig.sslgraph.dataset import get_node_dataset
from dig.sslgraph.method import pGRACE
from dig.sslgraph.utils.pgrace import generate_split

/home/grads/s/sairamana/miniconda3/envs/dig/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Loading dataset

In [2]:
from torch_geometric.datasets import Planetoid, CitationFull, WikiCS, Coauthor, Amazon
import torch_geometric.transforms as T
import torch
path = '/tmp/dig-experiments/'
dataset = Amazon(root=path, name='Photo', transform=T.NormalizeFeatures())
device = torch.device('cuda:0')
data = dataset[0].to(device=device)

In [3]:
split = generate_split(data.num_nodes, train_ratio = 0.1, val_ratio = 0.1)

#### Define encoders and contrastive model

You can refer to [https://github.com/CRIPAC-DIG/GRACE/blob/master/config.yaml](https://github.com/CRIPAC-DIG/GRACE/blob/master/config.yaml) for detailed training configs.

***Note***: Results in the GRACE paper uses different training-test splits to the public splits, due to which you may see different results in DIG and the original implementation of GRACE.

In [4]:
embed_dim = 256
encoder = Encoder(feat_dim=dataset[0].x.shape[1], hidden_dim=embed_dim, 
                  n_layers=2, gnn='gcn', act='relu', node_level=True, graph_level=False).to(device)
grace = pGRACE(dim=embed_dim, prob_edge_1 = 0.3, prob_edge_2 = 0.5, prob_feature_1 = 0.1, prob_feature_2 = 0.1, p_tau=0.7, tau=0.1).to(device)

In [7]:
evaluator = NodeUnsupervised(dataset, train_mask=split[0], test_mask = split[1], val_mask = split[2], device=0, log_interval=100)
evaluator.setup_train_config(p_lr=0.1, p_epoch=2000, p_weight_decay=1e-5, comp_embed_on='cuda:0')
evaluator.evaluate(learning_model=grace, encoder=encoder)

Pretraining: epoch 38:   2%|▏         | 37/2000 [00:16<14:33,  2.25it/s, loss=6.305740]


RuntimeError: size mismatch, got 745, 745x7650,7649